## **0. Download dataset**
**Note:** If you can't download using gdown due to limited number of downloads, please download it manually and upload it to your drive, then copy it from the drive to colab.
```python
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .
```

In [ ]:
# https://drive.google.com/file/d/1kU64ckjel-CpAH352bwU5hWMFT_caZxZ/view?usp=drive_link
!gdown --id 1kU64ckjel-CpAH352bwU5hWMFT_caZxZ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kU64ckjel-CpAH352bwU5hWMFT_caZxZ
To: /content/card_fraud_detection.zip
100% 69.2M/69.2M [00:01<00:00, 39.2MB/s]


In [ ]:
!unzip 'card_fraud_detection.zip'

Archive:  card_fraud_detection.zip
  inflating: creditcard.csv          


## **1. Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## **2. Read dataset**

In [ ]:
dataset_path = 'creditcard.csv'
df = pd.read_csv(
    dataset_path
)
df

In [ ]:
df.info()

In [ ]:
df.describe()

## **3. Add bias term**

In [ ]:
intercept = np.ones((X.shape[0], 1))

In [ ]:
X_b = np.concatenate((intercept, X), axis=1)

## **4. One-hot encoding label**

In [ ]:
n_classes = np.unique(y, axis=0).shape[0]
n_samples = y.shape[0]
y_encoded = np.array([np.zeros (n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1

## **5. Split train, val, test set**

In [ ]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X_b, y_encoded,
    test_size=val_size,
    random_state=random_state,
    shuffle=is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=test_size,
    random_state=random_state,
    shuffle=is_shuffle
)

In [ ]:
print(f'Number of training samples: {X_train.shape[0]}')
print(f'Number of val samples: {X_val.shape[0]}')
print(f'Number of test samples: {X_test.shape[0]}')

## **6. Normalization**

In [ ]:
normalizer = StandardScaler()
X_train[:, 1:] = normalizer.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalizer.transform(X_val[:, 1:])
X_test[:, 1:] = normalizer.transform(X_test[:, 1:])

In [ ]:
X_train

## **7. Define essential functions**

### **7.1. Softmax Function**

In [ ]:
def softmax(z):
    y_hat = np.exp(z) / np.exp(z).sum(axis=1)[:, None]

    return y_hat

### **7.2. Hypothesis function**

In [ ]:
def predict(X, theta):
    z = np.dot(X, theta)
    y_hat = softmax(z)

    return y_hat

### **7.3. Cross-entropy loss function**

In [ ]:
def compute_loss(y_hat, y):
    loss = -1 * (np.dot(y.T, np.log(y_hat))) / y.size

    return loss

### **7.4. Gradient function**

In [ ]:
def compute_gradient(X, y, y_hat):
    dtheta = np.dot(X.T, (y_hat - y)) / y.size

    return dtheta

### **7.5. Update weights function**

In [ ]:
def update_theta(theta, gradient, lr):
    theta = theta - lr * gradient

    return theta

### **7.6. Accuracy function**

In [ ]:
def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta)
    acc = (np.argmax(y_hat, axis=1) == np.argmax(y, axis=1)).mean()

    return acc

## **8. Training**

In [ ]:
lr = 0.01
epochs = 30
batch_size = 1024
n_features = X_train.shape[1]

np.random.seed(random_state)
theta = np.random.uniform(
    size=(n_features, n_classes)
)

In [ ]:
train_accs = []
train_losses = []
val_accs = []
val_losses = []

for epoch in range(epochs):
    train_batch_losses = []
    train_batch_accs = []
    val_batch_losses = []
    val_batch_accs = []

    for i in range(0, X_train.shape[0], batch_size):

        X_i = X_train[i: i + batch_size]
        y_i = y_train[i: i + batch_size]

        y_hat = predict(X_i, theta)
        train_loss = compute_loss(y_hat, y_i)
        dtheta = compute_gradient(X_i, y_i, y_hat)
        theta = update_theta(theta, dtheta, lr)
        train_batch_losses.append(train_loss)
        train_acc = compute_accuracy(X_train, theta, y_train)
        train_batch_accs.append(train_acc)

        y_val_hat = predict(X_val, theta)
        val_loss = compute_loss(y_val_hat, y_val)
        val_batch_losses.append(val_loss)
        val_acc = compute_accuracy(X_val, theta, y_val)
        val_batch_accs.append(val_acc)

    train_batch_loss = sum(train_batch_losses) / len(train_batch_losses)
    val_batch_loss = sum(val_batch_losses) / len(val_batch_losses)
    train_batch_acc = sum(train_batch_accs) / len(train_batch_accs)
    val_batch_acc = sum(val_batch_accs) / len(val_batch_accs)

    train_losses.append(train_batch_loss)
    train_accs.append(train_batch_acc)
    val_losses.append(val_batch_loss)
    val_accs.append(val_batch_acc)

    print(f'\nEPOCH {epoch + 1}:\tTraining Loss: {train_batch_loss:.3f}\tValidation Loss: {val_batch_loss:.3f}')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax[0, 0].plot(train_losses, color='green')
ax[0, 0].set(xlabel='Epoch', ylabel='Loss')
ax[0, 0].set_title('Training Loss')

ax[0, 1].plot(val_losses, color='orange')
ax[0, 1].set(xlabel='Epoch', ylabel='Loss')
ax[0, 1].set_title('Validation Loss')

ax[1, 0].plot(train_accs, color='green')
ax[1, 0].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 0].set_title('Training Accuracy')

ax[1, 1].plot(val_accs, color='orange')
ax[1, 1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 1].set_title('Validation Accuracy')

plt.show()

## **9. Evaluation**

In [ ]:
# Val set
val_set_acc = compute_accuracy(X_val, y_val, theta)
print('Evaluation on validation set:')
print(f'Accuracy: {val_set_acc}')

In [ ]:
# Test set
test_set_acc = compute_accuracy(X_test, y_test, theta)
print('Evaluation on test set:')
print(f'Accuracy: {test_set_acc}')